In [850]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import math
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv
/kaggle/input/house-prices-advanced-regression-techniques/train.csv


In [851]:
# Load into Pandas dataframe
train = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

# Create copies of train and test
train_c = train.drop(['Id'],axis=1)
test_c = test.drop(['Id'],axis=1)

- Data set information

In [852]:
train_c.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [853]:
test_c.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [854]:
# This is recommended by Author
train_c = train_c.drop(train_c[(train_c['GrLivArea']>4000)].index)

# Remove heavily skewed values
train_c = train_c[train_c['SalePrice'] < train_c['SalePrice'].quantile(0.95)]

- Check target skew

In [855]:
train_c['SalePrice'].skew()

0.6112894680812712

In [856]:
targets = train_c.SalePrice
train_c = train_c.drop(['SalePrice'],axis=1)

- concat train and test into single dataframe with labels

In [857]:
# Training set = 1, Test set = 0
train_c['DataType'] = 1 
test_c['DataType'] = 0
data = pd.concat([train_c, test_c], axis=0,sort=False)

In [858]:
data.head(-10)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,DataType
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,1
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,1
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,1
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,1
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444,20,NaN,125.0,31250,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,5,2006,WD,Normal,0
1445,90,RM,78.0,7020,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,11,2006,WD,Normal,0
1446,160,RM,41.0,2665,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,5,2006,WD,Normal,0
1447,20,RL,58.0,10172,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,10,2006,WD,Normal,0


In [859]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2842 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     2842 non-null   int64  
 1   MSZoning       2838 non-null   object 
 2   LotFrontage    2360 non-null   float64
 3   LotArea        2842 non-null   int64  
 4   Street         2842 non-null   object 
 5   Alley          198 non-null    object 
 6   LotShape       2842 non-null   object 
 7   LandContour    2842 non-null   object 
 8   Utilities      2840 non-null   object 
 9   LotConfig      2842 non-null   object 
 10  LandSlope      2842 non-null   object 
 11  Neighborhood   2842 non-null   object 
 12  Condition1     2842 non-null   object 
 13  Condition2     2842 non-null   object 
 14  BldgType       2842 non-null   object 
 15  HouseStyle     2842 non-null   object 
 16  OverallQual    2842 non-null   int64  
 17  OverallCond    2842 non-null   int64  
 18  YearBuil

- Check which features have many missing values

In [860]:
percentage = data.isna().sum()*100/data.shape[0]
percentage.sort_values(ascending=False).head()

PoolQC         99.718508
MiscFeature    96.305419
Alley          93.033075
Fence          80.049261
FireplaceQu    49.894441
dtype: float64

In [861]:
# Drop features with large number of missing values
data = data.drop(['Alley','PoolQC','Fence','MiscFeature'],axis=1)

- Split into numerical and categorical features

In [862]:
data_num = data.select_dtypes(exclude=['object'])
data_cat = data.select_dtypes(include=['object'])

# MSSubClass should be categorical
data_num = data_num.drop(['MSSubClass'],axis=1)
data_cat['MSSubClass'] = data['MSSubClass']

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


- Deal with numerical first

In [863]:
data_num.isnull().sum().sort_values(ascending=False)

LotFrontage      482
GarageYrBlt      159
MasVnrArea        22
BsmtFullBath       2
BsmtHalfBath       2
BsmtFinSF1         1
BsmtFinSF2         1
GarageCars         1
BsmtUnfSF          1
TotalBsmtSF        1
GarageArea         1
YearBuilt          0
YearRemodAdd       0
OverallCond        0
OverallQual        0
LotArea            0
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
DataType           0
YrSold             0
HalfBath           0
BedroomAbvGr       0
KitchenAbvGr       0
TotRmsAbvGrd       0
Fireplaces         0
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
3SsnPorch          0
ScreenPorch        0
PoolArea           0
MiscVal            0
MoSold             0
FullBath           0
dtype: int64

Create new features, fix up data

In [864]:
data_num['TotalPorch'] = data_num['OpenPorchSF'] + data_num['EnclosedPorch'] + data_num['3SsnPorch'] + data_num['ScreenPorch']
data_num['TotalFlrSF'] = data_num['1stFlrSF'] + data_num['2ndFlrSF']
data_num['TotalBsmtBath'] = data_num['BsmtFullBath'] + 0.5*data_num['BsmtHalfBath']
data_num['TotalBath'] = data_num['FullBath'] + 0.5*data_num['HalfBath']

data_num['YrSold'].loc[data_num['YrSold'] < data_num['YearBuilt']] = 2009
data_num['YearRemodAdd'].loc[data_num['YearBuilt'] > data_num['YearRemodAdd']] = 2002

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [865]:
# Replace these columns with median values
data_num['LotFrontage'].fillna(data_num['LotFrontage'].median(), inplace=True)
data_num['GarageYrBlt'].fillna(data_num['GarageYrBlt'].median(), inplace=True)

# Replace rest with just zeros
data_num.fillna(0, inplace=True)

In [866]:
data_num.skew()[abs(data_num.skew()) > 1].sort_values(ascending=False)

MiscVal          21.667125
PoolArea         18.473904
LowQualFinSF     12.254964
3SsnPorch        11.648248
LotArea          10.901027
KitchenAbvGr      4.238387
BsmtFinSF2        4.115899
EnclosedPorch     3.994294
ScreenPorch       3.960056
BsmtHalfBath      3.900475
MasVnrArea        2.674888
OpenPorchSF       2.605683
TotalPorch        2.268576
WoodDeckSF        1.866123
1stFlrSF          1.285832
LotFrontage       1.271525
dtype: float64

- Box Cox transformation for numerical data that are highly skewed

In [867]:
skewed = data_num.skew()[abs(data_num.skew()) > 0.75]
from scipy.special import boxcox1p
skewed_features = skewed.index
lam = 0.15
for feature in skewed_features:
    data_num[feature] = boxcox1p(data_num[feature], lam)

- Now deal with categorical data

In [868]:
data_cat.isnull().sum().sort_values(ascending=False)

FireplaceQu      1418
GarageCond        159
GarageQual        159
GarageFinish      159
GarageType        157
BsmtCond           82
BsmtExposure       82
BsmtQual           81
BsmtFinType2       80
BsmtFinType1       79
MasVnrType         23
MSZoning            4
Utilities           2
Functional          2
Electrical          1
KitchenQual         1
SaleType            1
Exterior2nd         1
Exterior1st         1
LandContour         0
BldgType            0
LotShape            0
Street              0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
MSSubClass          0
HouseStyle          0
RoofStyle           0
RoofMatl            0
ExterQual           0
ExterCond           0
SaleCondition       0
Heating             0
HeatingQC           0
CentralAir          0
PavedDrive          0
Foundation          0
dtype: int64

- Missing values

In [869]:
# Columns to fill with None
col = ['FireplaceQu','GarageQual','GarageCond','GarageFinish','GarageType','BsmtCond','BsmtQual','BsmtExposure','BsmtFinType1','BsmtFinType2']
data_cat[col] = data_cat[col].fillna('None')

# Columns to fill with most frequent
col = ['MasVnrType','MSZoning','Utilities','Functional','Electrical','SaleType','Exterior2nd','Exterior1st','KitchenQual']
data_cat[col] = data_cat[col].fillna(data_cat[col].mode().iloc[0])

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


- Ordinal Categorical data

In [870]:
mapper = {
    'Reg':4, 
    'IR1':3,
    'IR2':2,
    'IR3':1,
          
    'Lvl':4,
    'Bnk':3,
    'HLS':2,
    'Low':1,
    
    'AllPub':4,
    'NoSewr':3,
    'NoSeWa':2,
    'ELO':1,
    
    'Gtl':3,
    'Mod':2,
    'Sev':1,
    
    'Ex':5,
    'Gd':4,
    'TA':3,
    'Fa':2,
    'Po':1,
    'None':0,
    
    'GLQ':6,
    'ALQ':5,
    'BLQ':4,
    'Rec':3,
    'LwQ':2,
    'Unf':1,
    
    'SBrkr':5,
    'FuseA':4,
    'FuseF':3,
    'FuseP':2,
    'Mix':1,
    
    'Typ':8,
    'Min1':7,
    'Min2':6,
    'Mod':5,
    'Maj1':4,
    'Maj2':3,
    'Sev':2,
    'Sal':1,
    
    'Fin':3,
    'RFn':2,
    'Unf':1   
}
data_cat = data_cat.replace(mapper)

# Deal with PavedDrive seperately
pave = {"N" : 0, "P" : 1, "Y" : 2}
data_cat['PavedDrive'] = data_cat['PavedDrive'].replace(pave)

In [871]:
# Get dummies for rest of categorical data
data_cat_obj = data_cat.select_dtypes(include=['object'])
data_cat = pd.get_dummies(data_cat, columns=data_cat_obj.columns)
data_cat = pd.get_dummies(data_cat, columns=['MSSubClass'])
data_cat.head()

,LotShape,LandContour,Utilities,LandSlope,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtFinType1,BsmtFinType2,...,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSSubClass_120,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190
0,4,4,4,3,4,3,4,3,6,1,...,0,0,0,0,0,0,0,0,0,0
1,4,4,4,3,3,3,4,3,5,1,...,0,0,0,0,0,0,0,0,0,0
2,3,4,4,3,4,3,4,3,6,1,...,0,0,0,0,0,0,0,0,0,0
3,3,4,4,3,3,3,3,4,5,1,...,1,0,0,0,0,0,0,0,0,0
4,3,4,4,3,4,3,4,3,6,1,...,0,0,0,0,0,0,0,0,0,0


- concat all columns together

In [872]:
data_all = pd.concat([data_num, data_cat], axis=1,sort=False)

- Get train and targets for modelling

In [873]:
data_train = data_all[data_all['DataType'] == 1]
data_test = data_all[data_all['DataType'] == 0]

**Modeling Time**

In [874]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error

X_train,X_test,y_train,y_test = train_test_split(data_train,targets,test_size=0.25,random_state=42)

- Fit XGBoost

In [878]:
import xgboost as xgb
xgbr = xgb.XGBRegressor(n_estimators=1000,
                        learning_rate=0.01,
                        max_depth=4,
                        subsample=0.9,
                        colsample_bytree=0.8,
                        gamma=1,
                        random_state=42,
                        verbosity=0)
xgbr.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=1, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=4,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=0, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.9,
             tree_method='exact', validate_parameters=1, verbosity=0)

In [879]:
y_pred = xgbr.predict(X_test)
msle = math.sqrt(mean_squared_log_error(y_test, y_pred))
print("Mean Squared Log Error : ", msle)

Mean Squared Log Error :  0.1195585895964349


- Prepare submission file

In [877]:
submission = pd.DataFrame({"Id": test["Id"], "SalePrice": xgbr.predict(data_test)})
submission.to_csv('submission.csv', index=False)
submission

,Id,SalePrice
0,1461,119345.039062
1,1462,154843.656250
2,1463,189579.859375
3,1464,190725.625000
4,1465,183381.125000
...,...,...
1454,2915,80255.218750
1455,2916,76773.593750
1456,2917,158305.640625
1457,2918,112170.820312
